# Principaux modules d'OpenFisca

- OpenFisca-Core : moteur d'OpenFisca
- OpenFisca-France : législation socio-fiscale française
- OpenFisca-Web-API : API web (utilise OpenFisca-Core + OpenFisca-France)
- OpenFisca-Web-UI : Simulateur web d'OpenFisca (appelle OpenFisca-Web-API en HTTP)
- OpenFisca-France-Reform-Landais-Piketty-Saez : Un exemple de réforme

Les sources des modules : https://github.com/openfisca/

# Utilisation simplifiée d'OpenFisca

In [1]:
from datetime import date

import json

import openfisca_france
from openfisca_france.model.base import *

#importer numpy permet de coler deux sections de courbes (nécessaire quand on fait varier le salaire de base en-dessous du SMIC mensuel)
import numpy as np

# to debug / trace
from openfisca_core import tools
from openfisca_core.reforms import Reform

## Adaptation pour faciliter l'usage de ce notebook

_Ce correctif permet de redéfinir plusieurs fois la même variable sans provoquer d'erreur._

In [2]:
import functools

from openfisca_core.formulas import make_reference_formula_decorator
from openfisca_france.entities import entity_class_by_symbol

reference_formula = make_reference_formula_decorator(entity_class_by_symbol = entity_class_by_symbol, update = True)
reference_input_variable = functools.partial(reference_input_variable, update = True)

ImportError: cannot import name make_reference_formula_decorator

# Variable d'entrée

In [3]:
reference_input_variable(
    column = FloatCol,  # le type de variable : BoolCol, FloatCol, EnumCol, IntCol, StrCol...
    entity_class = Individus,  # Individus, Familles, FoyersFiscaux ou Menages
    label = "Salaire brut",
    name = 'salaire_brut',
    )

NameError: name 'reference_input_variable' is not defined

# Variable avec formule

In [4]:
@reference_formula
class salaire_net(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Salaire net"

    def function(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        salaire_brut = simulation.calculate('salaire_brut', period)

        return period, salaire_brut * 0.8

NameError: name 'reference_formula' is not defined

In [5]:
@reference_formula
class salaire_imposable(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Salaire imposable"

    def function(self, simulation, period):
        period = period.start.period(u'year').offset('first-of')
        salaire_net = simulation.calculate('salaire_net', period)

        return period, salaire_net * 0.9

NameError: name 'reference_formula' is not defined

# Variable avec différentes formules en fonction de la date

In [6]:
@reference_formula
class revenu_social_activite(DatedFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"RSA"

    @dated_function(date(2010, 1, 1), date(2010, 12, 31))
    def function_2010(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        salaire_net = simulation.calculate('salaire_net', period)
        age = simulation.calculate('age', period)

        return period, (age >= 18) * (salaire_net < 500) * 100.0

    @dated_function(date(2011, 1, 1))
    def function_2011_2014(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        salaire_net = simulation.calculate('salaire_net', period)
        age = simulation.calculate('age', period)

        return period, (age >= 18) * (salaire_net < 600) * 200.0

NameError: name 'reference_formula' is not defined

# Système socio-fiscal

In [7]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

NameError: name 'init_country' is not defined

# Simulation

In [8]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_brut = 12000,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_brut = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

NameError: name 'tax_benefit_system' is not defined

In [9]:
simulation.calculate('salaire_net')

NameError: name 'simulation' is not defined

In [10]:
simulation.calculate('salaire_imposable')

NameError: name 'simulation' is not defined

In [ ]:
simulation.calculate('salaire_net', '2014-12')

In [ ]:
simulation.calculate('revenu_social_activite', '2014-12')

In [ ]:
simulation.calculate('revenu_social_activite')

# Réforme

In [ ]:
from openfisca_core import reforms

In [ ]:
ReformeRevenuDeBase = reforms.make_reform(
    name = u"Réforme Revenu de base",
    reference = tax_benefit_system,
    )

In [ ]:
ReformeRevenuDeBase.input_variable(
    column = FloatCol,
    entity_class = Individus,  # Individus, Familles, FoyersFiscaux ou Menages
    label = u"Chômage brut",
    name = 'chomage_brut',
    )

In [ ]:
@ReformeRevenuDeBase.formula
class chomage_net(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Chômage net"

    def function(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        chomage_brut = simulation.calculate('chomage_brut', period)

        return period, chomage_brut * 0.8

In [ ]:
@ReformeRevenuDeBase.formula
class revenu_net(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Revenu net"

    def function(self, simulation, period):
        period = period.start.period(u'month').offset('first-of')
        chomage_net = simulation.calculate('chomage_net', period)
        salaire_net = simulation.calculate('salaire_net', period)

        return period, chomage_net + salaire_net

In [ ]:
reform = ReformeRevenuDeBase()

In [ ]:
reform_simulation = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_brut = 12000,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        chomage_brut = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
reform_simulation.calculate('salaire_net', '2014-12')

In [ ]:
reform_simulation.calculate('chomage_net', '2014-12')

In [ ]:
reform_simulation.calculate('revenu_net', '2014-12')